# Lab 9: Intro to SCIP

- [Example: Factory Production Optimization](#Example:-Factory-Production-Optimization)
- [Example: Investment Portfolio Optimization](#Example:-Investment-Portfolio-Optimization)

SCIP (Solving Constraint Integer Programs) is a high-performance solver for Mixed-Integer Linear Programming (MILP), Linear Programming (LP), and Constraint Programming (CP), developed at the Zuse Institute Berlin. It combines Branch-and-Bound, cutting planes, and heuristic methods to efficiently solve large-scale optimization problems. Widely used in academia and industry, SCIP is a free and flexible alternative to commercial solvers like Gurobi and CPLEX, making it ideal for applications in supply chain optimization, scheduling, and machine learning. Through its Python API (pyscipopt), SCIP integrates seamlessly with modern computational workflows.

In python, in order to use SCIP, you need to install the following package:
```bash
pip install pyscipopt
```



## Example: Factory Production Optimization

In this example, we solve a more complex production planning problem where a factory manufactures three products (A, B, and C). Each product requires labor hours, raw materials, and machine time, and the goal is to maximize total profit while respecting the available resources. The problem is formulated as a Mixed-Integer Linear Program (MILP), where the decision variables (number of units produced) are integer values. SCIP efficiently solves the problem using Branch-and-Bound and cutting-plane methods.

The problem is defined as follows:

A factory produces three products (A, B, and C) with the following characteristics:

| Product | Profit per Unit | Labor | Raw Material | Machine Time |  
| ----------- | ---------------- | ----------------- | ------------------------ | ------------------------ |  
| A       | 7                | 2                 | 3                        | 1                        |  
| B      | 5                | 3                 | 2                        | 2                        |  
| C       | 4                | 1                 | 2                        | 2                        |  

The factory has total resource constraints:
- Labor: 15 hours
- Material: 10 units
- Machine Time: 10 hours

The objective is to determine the optimal number of units of each product to produce to maximize total profit, while ensuring resource limits are not exceeded.


In [2]:
from pyscipopt import Model

# Create SCIP model
model = Model("Factory Production Optimization")

# Define decision variables (integer: number of units produced)
A = model.addVar("A", vtype="I", lb=0)
B = model.addVar("B", vtype="I", lb=0)
C = model.addVar("C", vtype="I", lb=0)

# Set objective function: Maximize profit
model.setObjective(7 * A + 5 * B + 4 * C, "maximize")

# Add constraints for resources
model.addCons(2 * A + 3 * B + 1 * C <= 15, "Labor Constraint")        # Labor constraint
model.addCons(3 * A + 2 * B + 2 * C <= 10, "Material Constraint")     # Raw material constraint
model.addCons(1 * A + 2 * B + 2 * C <= 10, "Machine Time Constraint") # Machine time constraint

# Solve model
model.optimize()

# Print solution
if model.getStatus() == "optimal":
    print("Optimal Production Plan:")
    print(f"Produce {int(model.getVal(A))} units of Product A")
    print(f"Produce {int(model.getVal(B))} units of Product B")
    print(f"Produce {int(model.getVal(C))} units of Product C")
    print(f"Maximum Profit: {model.getObjVal()}")
else:
    print("No optimal solution found.")

Optimal Production Plan:feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 5 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, exhaustive) 0 del vars, 1 del conss, 0 add conss, 5 chg bounds, 1 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (3 rounds: 3 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 1 deleted constraints, 0 added constraints, 5 tightened bounds, 0 added holes, 1 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 3 variables (0 bin, 3 int, 0 impl, 0 cont) and 2 constraints
      2 constraints of type <linear>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00
transformed 1/1 original solutions t

The SCIP solver successfully found the optimal production plan, maximizing the factory’s profit while satisfying resource constraints. The solver determined that the best strategy is to produce 5 units of Product B, while producing 0 units of Products A and C. This results in a maximum profit of $25. The solution was found efficiently, with SCIP identifying the optimal result in 0.00 seconds using heuristic methods and presolving techniques, indicating a relatively simple problem structure.

From the solver’s output, we can see that the primal and dual bounds are equal (both at 25.0), and the optimality gap is 0%, confirming that this is indeed the best possible solution. The presolving step reduced the problem by simplifying constraints and tightening bounds, making it computationally efficient. The choice of producing only Product B suggests that it provides the highest profit relative to its resource consumption, making it the most economical option under the given constraints.

## Example: Investment Portfolio Optimization

A company has **$10,000 to invest** in three different assets (**Stocks, Bonds, and Real Estate**). Each investment has an expected **return on investment (ROI)** and a **risk factor**. The goal is to **maximize total ROI while keeping the total risk below a certain threshold**.

The available assets have the following characteristics:

| Investment | Expected ROI | Risk Factor | Minimum Investment | Maximum Investment |  
|-------|------------------|-------------|------------------|------------------|  
| Stocks | 12               | 8           | 2,000            | 6,000            |  
| Bonds | 5                | 2           | 1,000            | 5,000            |  
| Real Estate | 8                | 5           | 3,000            | 7,000            |  

The company wants to **maximize ROI** while ensuring the **total risk factor does not exceed 20**.

In [3]:
# Create SCIP model
model = Model("Investment Portfolio Optimization")

# Define decision variables (amount invested in each asset)
Stocks = model.addVar("Stocks", vtype="C", lb=2000, ub=6000)
Bonds = model.addVar("Bonds", vtype="C", lb=1000, ub=5000)
RealEstate = model.addVar("RealEstate", vtype="C", lb=3000, ub=7000)

# Set objective function: Maximize ROI
model.setObjective(0.12 * Stocks + 0.05 * Bonds + 0.08 * RealEstate, "maximize")

# Add constraints
model.addCons(Stocks + Bonds + RealEstate <= 10000, "Total Investment Constraint")  # Budget constraint
model.addCons(8 * Stocks + 2 * Bonds + 5 * RealEstate <= 20 * 10000, "Risk Constraint")  # Risk constraint

# Solve model
model.optimize()

# Print solution
if model.getStatus() == "optimal":
    print("Optimal Investment Strategy:")
    print(f"Invest ${model.getVal(Stocks):.2f} in Stocks")
    print(f"Invest ${model.getVal(Bonds):.2f} in Bonds")
    print(f"Invest ${model.getVal(RealEstate):.2f} in Real Estate")
    print(f"Maximum ROI: ${model.getObjVal():.2f}")
else:
    print("No optimal solution found.")

Optimal Investment Strategy:feasible solution found by trivial heuristic after 0.0 seconds, objective value 5.300000e+02
presolving:

Invest $6000.00 in Stocks
Invest $1000.00 in Bonds
Invest $3000.00 in Real Estate
Maximum ROI: $1010.00
(round 1, fast)       0 del vars, 1 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 3 deleted vars, 2 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/2 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : +1.01000000000000e+03 (2 solutions)
Dual Bound         : +1.01000000000000e+03
Gap                : 0.00 %


SCIP successfully found the optimal investment strategy, allocating \\$6,000 to Stocks, \\$1,000 to Bonds, and \\$3,000 to Real Estate, yielding a maximum return on investment (ROI) of \\$1,010. The solver identified this solution efficiently, with zero solving nodes and an optimality gap of 0%, meaning the result is proven to be the best possible allocation under the given constraints. The primal and dual bounds match, confirming that no better feasible solution exists.

The allocation strategy indicates that stocks offer the highest return per dollar within the allowed risk limits, which is why SCIP allocated the maximum permissible amount (\\$6,000) to them. The remaining budget is spread across Real Estate (\\$3,000) and Bonds (\\$1,000) to ensure the total investment remains within \\$10,000 while keeping risk under control. The solver’s presolving phase removed redundant constraints and variables, making the problem computationally efficient to solve. This result highlights how optimization models can be used in financial decision-making, providing a structured approach to maximizing returns while considering risk constraints. 